<a href="https://colab.research.google.com/github/Gaussiandra/RuREBus_NER_RE/blob/master/hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Aug 16 09:54:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

global_path = '/content/drive/My Drive/ML/NLP sber/8/RuREBus/'
global_repo_path = global_path+'data/RuREBus-master/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%writefile requirements.txt
razdel
ipymarkup
pytorch-crf
tensorflow-gpu==1.15.2
deeppavlov

Writing requirements.txt


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 411.0MB 42kB/s 
     |████████████████████████████████| 942kB 33.2MB/s 
     |████████████████████████████████| 3.8MB 50.8MB/s 
     |████████████████████████████████| 512kB 50.6MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 7.3MB 8.3MB/s 
     |████████████████████████████████| 665kB 50.8MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 1.5MB 49.7MB/s 
     |████████████████████████████████| 8.0MB 28.3MB/s 
     |████████████████████████████████| 2.1MB 52.0MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 860kB 54.6MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 10.4MB 165kB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 512kB 55.5MB/s 
    

In [2]:
import numpy as np
import torch
import os
import razdel
import ipymarkup
import pickle

from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from collections import namedtuple
from torchcrf import CRF
from zipfile import ZipFile
from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


In [7]:
# for i in range(1, 4):
#     ZipFile(global_repo_path+f'train_data/train_part_{i}.zip', 'r').extractall(
#             global_repo_path+f'train_data/train_parts/')

# ZipFile(global_repo_path+f'test_data/test_full.zip', 'r').extractall(
#         global_repo_path+f'test_data/test_parts/')

In [3]:
DocRepr = namedtuple('DocRepr', 'text ann')
WordRepr = namedtuple('WordRepr', 'raw_text tokens char_ids ner_spans labels file_name elmo_embedding')

class Task1Dataset(Dataset):
    def __init__(self, folder_path, elmo_dim=None, batch_size=3, is_train=True,
                 custom_ent2idx=None, custom_char2idx=None, dummy_obj=False):
        if dummy_obj:
            return

        self.folder_path = folder_path
        self.elmo_dim = elmo_dim

        self.docs = {}
        for dirpath, dirnames, filenames in os.walk(self.folder_path):
            for extension_file_name in filenames:
                filename = extension_file_name[:extension_file_name.rfind('.')]
                try:
                    self.docs[filename] = DocRepr(
                        open(os.path.join(dirpath, filename+'.txt')).read(),
                        open(os.path.join(dirpath, filename+'.ann')).read(),
                    )
                except FileNotFoundError:
                    print(f'"{filename}" was ignored.')                       

        self.samples = []
        self.ent2idx = {'OUT': 0} if is_train else custom_ent2idx
        self.char2idx = {'<pad>': 0, '<unk>': 1} if is_train else custom_char2idx
        for name, doc in tqdm(self.docs.items()):
            tokens =  list(razdel.tokenize(doc.text))

            ner_spans = [i.split('\t') for i in doc.ann.split('\n')]
            for i in range(len(ner_spans)):
                if ner_spans[i][0].startswith('T'):
                    entity, start, stop = ner_spans[i][1].split()
                    ner_spans[i][1] = (entity, int(start), int(stop))

            labels = []
            for token in tokens:
                label = self.ent2idx['OUT']
                for span in ner_spans:
                    if span[0].startswith('T'):
                        entity, start, stop = span[1]
                        if 'B-'+entity not in self.ent2idx and is_train:
                            self.ent2idx['B-'+entity] = len(self.ent2idx)
                            self.ent2idx['I-'+entity] = len(self.ent2idx)

                        if token.start == start:
                            label = self.ent2idx['B-'+entity]
                        elif token.start > start and token.stop <= stop:
                            label = self.ent2idx['I-'+entity]
                labels.append(label)

            char_ids = []
            for token in tokens:
                char_word_repr = []
                for char in token.text:
                    if char not in self.char2idx and is_train:
                        self.char2idx[char] = len(self.char2idx)
                    if not is_train and char not in self.char2idx:
                        char_word_repr.append(self.char2idx['<unk>'])
                    else:
                        char_word_repr.append(self.char2idx[char])
                char_ids.append(char_word_repr)

            self.samples.append(WordRepr(
                doc.text,
                tokens,
                char_ids,
                ner_spans,
                labels,
                name,
                None,
            ))
        
        self.elmo_embedder = ELMoEmbedder( #
            "http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz", 
            elmo_output_names=["elmo"], 
            dim=self.elmo_dim
        )

        for batch_num in tqdm(range(0, len(self.samples), batch_size)):
            embeddings = self.elmo_embedder([
                [token.text for token in sample.tokens] 
                for sample in self.samples[batch_num:batch_num+batch_size]
            ])
            for i, emb in enumerate(embeddings):
                self.samples[batch_num+i] = self.samples[batch_num+i]._replace(elmo_embedding = emb)

    def __getitem__(self, idx):
        return (
            self.samples[idx].elmo_embedding,
            self.samples[idx].char_ids,
            self.samples[idx].labels
        )
            
    def __len__(self):
        return len(self.samples)

In [4]:
task1_train_ds = Task1Dataset(global_repo_path+'train_data/train_parts')
task1_test_ds = Task1Dataset(global_repo_path+'test_data/test_parts',
                             is_train=False,
                             custom_ent2idx=task1_train_ds.ent2idx,
                             custom_char2idx=task1_train_ds.char2idx)
len(task1_train_ds), len(task1_train_ds.ent2idx), len(task1_train_ds.char2idx)

"" was ignored.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(188, 17, 179)

In [5]:
# pickle.dump(task1_train_ds.samples, open(global_repo_path+'ds1_train_dim1024.pkl', 'wb'))
# pickle.dump(task1_train_ds.ent2idx, open(global_repo_path+'ds1_ent2idx1024.pkl', 'wb'))
# pickle.dump(task1_train_ds.char2idx, open(global_repo_path+'ds1_char2idx1024.pkl', 'wb'))

# pickle.dump(task1_test_ds.samples, open(global_repo_path+'ds1_test_dim1024.pkl', 'wb'))

In [4]:
task1_train_ds = Task1Dataset(None, dummy_obj=True)
task1_train_ds.samples = pickle.load(open(global_repo_path+'ds1_train_dim1024.pkl', 'rb'))

task1_test_ds = Task1Dataset(None, dummy_obj=True)
task1_test_ds.samples = pickle.load(open(global_repo_path+'ds1_test_dim1024.pkl', 'rb'))
task1_test_ds.ent2idx = pickle.load(open(global_repo_path+'ds1_ent2idx1024.pkl', 'rb'))
task1_test_ds.char2idx = pickle.load(open(global_repo_path+'ds1_char2idx1024.pkl', 'rb'))

In [42]:
del model1
torch.cuda.empty_cache()
import gc
gc.collect()

856

In [5]:
for i in task1_test_ds.samples[2]:
    print(i)

ГОСУДАРСТВЕННАЯ ПРОГРАММА
РЕСПУБЛИКИ КОМИ "ЮСТИЦИЯ И ОБЕСПЕЧЕНИЕ
ПРАВОПОРЯДКА В РЕСПУБЛИКЕ КОМИ"


ПАСПОРТ
Государственной программы Республики Коми "Юстиция
и обеспечение правопорядка в Республике Коми"
(далее - Программа)


Паспорт подпрограммы
"Правовая защищенность населения Республики Коми"
(далее - Подпрограмма 1)


Паспорт подпрограммы
"Правопорядок"
(далее - Подпрограмма 2)


Паспорт подпрограммы
"Государственная регистрация актов
гражданского состояния в Республике Коми"
(далее - Подпрограмма 3)


Паспорт подпрограммы
"Мировые судьи в Республике Коми"
(далее - Подпрограмма 4)


Паспорт подпрограммы
"Обеспечение реализации Программы"
(далее - Подпрограмма 5)


Приоритеты и цели реализуемой в Республике Коми
государственной политики, общая характеристика участия
муниципальных образований в Республике Коми
в реализации государственной программы

Основная цель и задачи Программы соответствуют приоритетам, целям и основным направлениям социально-экономического развития Республики К

In [6]:
def pad_seq(batch):
    emb = [torch.tensor(sample[0]) for sample in batch]

    max_word_len = max([len(word) for sample in batch for word in sample[1]])
    max_word_len = min(50, max_word_len)
    char_list = []
    for sample in batch:
        words = []
        for word in sample[1]:
            words.append(word[:max_word_len] + [0]*(max_word_len-len(word)))
        char_list.append(torch.tensor(words))

    labels = [torch.tensor(sample[2]) for sample in batch]

    padded_emb = pad_sequence(emb, batch_first=True)
    padded_chars = pad_sequence(char_list, batch_first=True)
    padded_labels = pad_sequence(labels, batch_first=True)

    return padded_emb, padded_chars, padded_labels

In [17]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 6

train_iter = DataLoader(task1_train_ds, batch_size, collate_fn=pad_seq, shuffle=True)
test_iter = DataLoader(task1_test_ds, batch_size,  collate_fn=pad_seq)

In [8]:
next(iter(train_iter))

(tensor([[[-1.2544, -0.0240, -1.1977,  ...,  0.1366,  0.6973, -0.1697],
          [-0.1308,  0.6212, -0.2699,  ..., -1.3370,  0.0717, -0.2458],
          [-0.8376, -0.1626,  0.2736,  ...,  0.7278,  1.1011, -0.9911],
          ...,
          [-0.9042, -0.5032, -1.5277,  ..., -0.1364,  0.2035, -0.7817],
          [-0.0681, -0.1358, -1.6670,  ..., -0.2093,  0.3832,  0.2251],
          [-1.1541, -1.6579, -1.3477,  ..., -0.2447,  0.2120, -0.3561]],
 
         [[-1.6971, -0.5223, -0.8536,  ...,  1.0573,  1.3189, -0.6585],
          [-0.8628, -0.0551, -0.8826,  ...,  0.6902,  1.2680, -0.5310],
          [-1.1935,  0.3731, -1.5127,  ...,  0.9916,  1.2035, -0.9274],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]]),
 tensor([[[ 5,  9, 10,  ...,  0,  0,  0],
          [35, 24, 25,  ...,  0,  0,  0],
          [ 7, 2

In [9]:
for i in _8:
    print(i.shape)

torch.Size([2, 2079, 1024])
torch.Size([2, 2079, 24])
torch.Size([2, 2079])


In [12]:
class NERModel(nn.Module):
    def __init__(self, n_classes=17, n_chars=179):
        super().__init__()

        self.char_embedding = nn.Embedding(n_chars, 250)
        convolution_params = [(3, 75), (4, 75), (5, 75), (4, 75), (7, 50)]
        self.conv_list = nn.ModuleList([nn.Conv2d(1, n, (k, 250)) for k, n in convolution_params])
        self.lstm = nn.LSTM(1374, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
        self.dropout_layer = nn.Dropout(0.1)
        self.output_layer = nn.Linear(2*256, n_classes)

    def forward(self, elmo_emb, chars):
        char_emb = self.char_embedding(chars)
        
        conv_res = []
        for f in self.conv_list:
            convolved_res = f(char_emb.reshape(-1, 1, char_emb.shape[2], char_emb.shape[3]))
            conv_res.append(convolved_res.max(dim=2)[0].squeeze(2).reshape(*char_emb.shape[:2], f.out_channels))
        conv_res = torch.cat(conv_res, dim=2)
        
        elmo_cnn_features = torch.cat([elmo_emb.detach(), conv_res], dim=2)
        x, _ = self.lstm(elmo_cnn_features)
        x = self.dropout_layer(x)
        x = self.output_layer(x)
        return x

model1 = NERModel().to(device)

In [13]:
loss_fn = CRF(17, batch_first=True).to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=0.001)
losses = []
n_iter = 0

print(sum(p.numel() for p in model1.parameters()))

5360617


In [44]:
for epoch in range(10):
    for elmo_emb, chars, y in tqdm(train_iter):
        optimizer.zero_grad()
        model1.train()

        elmo_emb, chars, y = elmo_emb.to(device), chars.to(device), y.to(device)
        y_pred = model1(elmo_emb, chars)

        loss = -loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        n_iter += 1
        if n_iter % 5 == 0:
            print(n_iter, np.mean(losses[-5:]))

1245 6754.546875
1250 7026.42021484375
1255 7041.39462890625
1260 9073.21162109375
1265 7674.3138671875
1270 5630.307421875



1275 5289.587902832031
1280 5475.62939453125
1285 8405.668408203124
1290 8954.38994140625
1295 4656.448974609375
1300 6330.4255859375



1305 6624.05068359375
1310 8345.5287109375
1315 5126.3140625
1320 6423.93642578125
1325 6559.55546875
1330 5992.600927734375
1335 8984.26357421875



1340 7086.0166015625
1345 5134.26640625
1350 8512.61455078125
1355 6385.37021484375
1360 8531.54814453125
1365 6013.905078125



1370 6119.836328125
1375 4950.22841796875
1380 5407.66923828125
1385 8929.01533203125
1390 6475.98369140625
1395 8031.57783203125
1400 7820.695825195313



...

1435 4636.7532958984375
1440 6720.98603515625
1445 6675.6173828125
1450 5218.344775390625
1455 10379.92158203125
1460 5270.02685546875



1465 7992.799365234375
1470 6180.928515625
1475 7076.9443359375
1480 6322.6140625
1485 9386.58251953125
1490 7052.34365234375
1495 6962.116015625



1500 5834.27490234375
1505 7484.64296875
1510 6782.43037109375
1515 5236.33017578125
1520 8047.44609375
1525 8609.9359375



1530 3729.4567626953126
1535 6300.5162109375


KeyboardInterrupt: ignored

In [43]:
#pickle.dump(model1, open(global_repo_path+'model1_1024.pkl', 'wb'))

model1 = pickle.load(open(global_repo_path+'model1_1024.pkl', 'rb'))
model1.lstm.flatten_parameters()

In [45]:
final_predicts = []
for batch_idx, (elmo_emb, chars, y) in enumerate(tqdm(test_iter)):
    model1.eval()

    elmo_emb, chars, y = elmo_emb.to(device), chars.to(device), y.to(device)
    y_pred = model1(elmo_emb, chars)

    for sample_idx, sample in enumerate(y_pred):
        cur_len = len(task1_test_ds.samples[batch_idx*batch_size + sample_idx].tokens)
        final_predicts.append(torch.argmax(sample[:cur_len], dim=1).cpu().numpy())

In [22]:
import os, sys
lib_path = os.path.abspath(os.path.join(global_repo_path, 'eval_scripts'))
sys.path.append(lib_path)
import evaluate_ners, brat_format

In [23]:
idx2ent = {}
for k, v in task1_test_ds.ent2idx.items():
    idx2ent[v] = k

In [46]:
n_ners = 0
for sample_num, sample in enumerate(tqdm(final_predicts)):
    orig_sample = task1_test_ds.samples[sample_num]
    brat_doc = brat_format.BratDoc(orig_sample.raw_text)

    # Убираем неконсистентность
    out_idx = task1_test_ds.ent2idx['OUT']
    for token_num, token in enumerate(sample):
        if idx2ent[token].startswith('I-'):
            if token_num == 0:
                sample[token_num] = out_idx
            elif sample[token_num - 1] == out_idx:
                sample[token_num] = out_idx

    brat_spans = []
    for token_num, token in enumerate(sample):
        if idx2ent[token].startswith('B-'):
            brat_spans.append([
                idx2ent[token][2:], 
                orig_sample.tokens[token_num].start,
                orig_sample.tokens[token_num].stop,
            ])
        elif idx2ent[token].startswith('I-') and len(brat_spans) and brat_spans[-1][0] == idx2ent[token][2:]:
            brat_spans[-1][2] =  orig_sample.tokens[token_num].stop

    for span in brat_spans:
        n_ners += 1
        brat_doc.add_ner(n_ners, *span)

    brat_doc.write_to_file(global_repo_path+f'test_data/pred_parts/{orig_sample.file_name}.ann')
    #assert False

In [25]:
evaluate_ners.calc_ner_f1(global_repo_path+f'test_data/test_parts/', 
                          global_repo_path+f'test_data/pred_parts/')

0.4928182807399347